In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm, tqdm_notebook

from scipy import stats
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data
#from transformers import *
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
import os
import re
import math
import random
from matplotlib import pyplot as plt
import warnings
from math import floor, ceil

warnings.filterwarnings('ignore')
device = torch.device('cuda')
torch.backends.cudnn.benchmark=True

%matplotlib inline

In [2]:
train = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/train.csv').fillna(' ')
test = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/test.csv').fillna(' ')
sub = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/sample_submission.csv').fillna(' ')

In [3]:
train.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [4]:
def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False
                current_segment_id = 1#新增 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512-1, 
                t_max_len=70-1, q_max_len=219, a_max_len=219):#???

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

        t = t[:t_new_len]
        q = norm_token_length(q, q_new_len)
        a = norm_token_length(a, a_new_len)
    
    return t, q, a

def norm_token_length(tokens, l):
    if len(tokens) > l:
        head = l//2
        tail = l - head
        return tokens[:head] + tokens[-tail:]
    else:
        return tokens[:l]

def _convert_to_bert_inputs(title, question, answer, cate, max_sequence_length=512):
    """Converts tokenized input to ids, masks and segments for BERT"""
    #stoken = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
    stoken_1 = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"]
    stoken_2 = ["[CLS]"] + title + ["[SEP]"] + answer + ["[SEP]"]
    input_ids = _get_ids(stoken_1, tokenizer, max_sequence_length)
    input_ids_2 = _get_ids(stoken_2, tokenizer, max_sequence_length)
    input_segments = _get_segments(stoken_1, max_sequence_length)
    input_segments_2 = _get_segments(stoken_2, max_sequence_length)
    
    #return [input_ids, input_segments]
    return input_ids, input_segments,input_ids_2,input_segments_2

def convert_row(row,pretrained_weights):
    #c = f"[{row['category'].lower()}]"

    if pretrained_weights == "bert-base-uncased":
        c = f"[{row['category'].lower()}]"#type:str
    elif pretrained_weights == "bert-base-cased":
        c = f"[{row['category']}]"#type:str
    elif pretrained_weights == "xlnet-base-cased":
        c = f"[{row['category']}]"#type:str
    t, q, a = title = row["question_title"], row["question_body"], row["answer"]#type:str


    t, q, a = row["question_title"], row["question_body"], row["answer"]
    t, q, a = _trim_input(t, q, a)
    #ids, segments = _convert_to_bert_inputs(t, q, a, c)
    ids, segments, ids2, segments2 = _convert_to_bert_inputs(t, q, a, c)
    #total_input=[np.array([[ids, segments]]),np.array([[ids2, segments2]])]
    # total_input=[]
    # print(np.array([[ids, segments]]).shape)
    # total_input.append(np.array([[ids, segments]]))
    # total_input.append(np.array([[ids2, segments2]]))
    # return total_input
    return np.array([[ids, segments, ids2, segments2]])

In [5]:
#model="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained("./token_model_config/tokenizer/")
#tokenizer.add_tokens("./bert_based_tokenizer/added_tokens.json")
#tokenizer.add_tokens(categories)


In [6]:
len(tokenizer)

29001

In [9]:
def custom_loss(data,data2, targets,targets2):
    
    # mse = nn.MSELoss(reduction="none")(data[:,:30].sigmoid(), targets[:,:30])
    # bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:30], targets[:,:30])#??
    mse = nn.MSELoss(reduction="none")(data[:,:].sigmoid(), targets[:,:])
    bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:], targets[:,:])#??
    
    mse2 = nn.MSELoss(reduction="none")(data2[:,:].sigmoid(), targets2[:,:])
    bce2 = nn.BCEWithLogitsLoss(reduction='none')(data2[:,:], targets2[:,:])#??
    #w =  targets[:,30:]
    #loss = (mse*w).sum() + bce.sum()
    loss = (mse).sum()+ bce.sum()+mse2.sum()+bce2.sum()
    return loss

class CustomBert(nn.Module):
    def __init__(self, config_path=None):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(config_path) 
        
        self.config.Q_labels = 21
        self.config.A_labels = 9
        self.config.output_hidden_states = True
        self.n_use_layer = 4 #原本
        #self.n_use_layer = 2
        self.double_bert= 1
        self.n_labels = self.config.num_labels
        #self.config.save_pretrained("bert_based_config")
        #self.config.save_pretrained(output_dir+"config")
        #self.bert = BertModel(config)
        self.bert=AutoModel.from_config(self.config)
        self.bert2=AutoModel.from_config(self.config)
        #self.bert.save_pretrained('bert_based_model')
        #self.bert.save_pretrained(output_dir+"model")
        # self.dense1 = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.hidden_size*self.n_use_layer)
        # self.dense2 = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.hidden_size*self.n_use_layer)
        # self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        # self.classifier = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.num_labels)

        self.dense1 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.double_bert*self.config.hidden_size*self.n_use_layer)
        self.dense2 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.double_bert*self.config.hidden_size*self.n_use_layer)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.config.Q_labels)
        self.classifier2 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.config.A_labels)
        #self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None
                ,input_ids2=None, attention_mask2=None, token_type_ids2=None,position_ids=None, head_mask=None, inputs_embeds=None, labels=None):

        # outputs = self.bert(input_ids,
        #                     attention_mask=attention_mask,
        #                     token_type_ids=token_type_ids,
        #                     position_ids=position_ids,
        #                     head_mask=head_mask,
        #                     inputs_embeds=inputs_embeds)
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                            )
        outputs2 = self.bert2(input_ids2,
                            attention_mask=attention_mask2,
                            token_type_ids=token_type_ids2
                            )
        
        #print(outputs[2][-1].shape)
        pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後4個layer的cls output concat在一起，把4個(8,768) concat，變成(8,3072) #原本
        pooled_output2 = torch.cat([outputs2[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)
        #pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後2個layer的cls output concat在一起,把2個(8,768) concat，變成(8,1536)
        #pooled_output2 = torch.cat([outputs2[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#同上
        #double_pooled_output=torch.cat([pooled_output,pooled_output],dim=1)#(8,3072)
        
        pooled_output = self.dense1(pooled_output)
        pooled_output = self.dense2(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        pooled_output2 = self.dense1(pooled_output2)
        pooled_output2 = self.dense2(pooled_output2)
        pooled_output2 = self.dropout(pooled_output2)
        logits2 = self.classifier2(pooled_output2)

        # double_pooled_output = self.dense1(double_pooled_output)
        # double_pooled_output = self.dense2(double_pooled_output)
        # double_pooled_output = self.dropout(double_pooled_output)
        # logits = self.classifier(double_pooled_output)

        outputs = (logits,) + outputs[2:]
        outputs2 = (logits2,) + outputs2[2:]

        return outputs,outputs2

In [10]:
model=CustomBert("token_model_config/config/config.json")


In [12]:
BS=8

In [13]:
pretrained_weights = 'bert-base-cased'
#X_test = test[["question_title", "question_body", "answer", "category"]].progress_apply(lambda x: convert_row(x, pretrained_weights), axis=1).values#shape(476)
X_test = test[["question_title", "question_body", "answer", "category"]].apply(lambda x: convert_row(x, pretrained_weights), axis=1).values#shape(476)
#np.vstack(X_test).shape : (476, 2, 512)
X_test = np.vstack(X_test).reshape((len(X_test), 2048))

test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BS, shuffle=False)

In [11]:
len(tokenizer)

29001

In [12]:
X_test.shape

(476, 2048)

In [14]:
model = model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.bert.resize_token_embeddings(len(tokenizer))#??
model.eval()

CustomBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29001, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (d

In [17]:
#model_dir_target = "../input/Optimize binning/bert-base-cased"#

cased_pred_lst = []
for fold in range(6):
    # if fold in [0,1,2,3,4,5,6,7]:
    #     continue
    #bert_path = f"{model_dir_target}/bert-base-cased_f{fold}_best"
    #bert_path=f"./DoubleBertBasedCase/bce_no_opt_binning/double-bert-based-case_f{fold}_best"
    bert_path=f"./Bce-NoOptbinning/double-bert-based-case_f{fold}_best"
    model.load_state_dict(torch.load(bert_path),strict=False)
    
    lst = []
    for i, (x_batch,)  in enumerate(test_loader):
        # input_ids = x_batch[:, :512]
        # token_ids = x_batch[:, 512:]
        input_ids = x_batch[:, :512]
        token_ids = x_batch[:, 512:1024]
        input_ids2 = x_batch[:, 1024:1536]
        token_ids2 = x_batch[:, 1536:]
        #pred = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device))
        pred, pred2 = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device),
                     input_ids2=input_ids2.to(device),attention_mask2=(input_ids2 > 0).to(device),token_type_ids2=token_ids2.to(device))
        total_y_pred=torch.cat((pred[0],pred2[0]),dim=1)
        lst.append(total_y_pred.detach().cpu().squeeze().numpy())
    test_pred = np.vstack(lst)#shape:(476, 30)
    
    cased_pred_lst.append(test_pred)

In [18]:
cased_pred_lst[0].shape

(476, 30)

In [16]:
np.array(cased_pred_lst).mean(0)

array([[ 2.9256349 ,  0.41644263, -2.4063394 , ..., -3.5992382 ,
         2.9604666 ,  3.010326  ],
       [ 2.0363727 , -0.09338026, -6.9384775 , ..., -3.139693  ,
        -3.6852486 ,  1.8472881 ],
       [ 2.9483905 ,  0.65646565, -5.5941467 , ..., -3.1242855 ,
         2.5920534 ,  2.9166663 ],
       ...,
       [ 1.6984911 , -0.61168987, -6.0037622 , ..., -1.9840653 ,
         0.5307194 ,  2.5947602 ],
       [ 2.7981777 ,  1.365348  , -4.0786796 , ..., -3.3693376 ,
         1.8809958 ,  3.0543346 ],
       [ 2.278844  , -0.09467044, -4.025428  , ..., -3.0977066 ,
        -2.8133788 ,  2.2528353 ]], dtype=float32)

In [19]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
cased_pred = np.array(cased_pred_lst).mean(0)
cased_pred = sigmoid(cased_pred)
cased_pred

array([[0.9494082 , 0.63013375, 0.14897776, ..., 0.00253813, 0.9804977 ,
        0.9521161 ],
       [0.808026  , 0.412042  , 0.00171434, ..., 0.01682095, 0.00647272,
        0.90910053],
       [0.9450579 , 0.67390245, 0.01062978, ..., 0.002915  , 0.9784374 ,
        0.9000374 ],
       ...,
       [0.8384446 , 0.40093726, 0.01027456, ..., 0.21919301, 0.4604722 ,
        0.90800893],
       [0.9636179 , 0.90189177, 0.02156096, ..., 0.02346952, 0.97254926,
        0.9802336 ],
       [0.9293306 , 0.430232  , 0.02866988, ..., 0.07433899, 0.13348521,
        0.8728599 ]], dtype=float32)

In [18]:
sub[sub.columns[1:]] = cased_pred
sub.to_csv("submission.csv", index=False)
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.949099,0.602632,0.082691,0.425644,0.664275,0.655471,0.707268,0.669800,0.798551,...,0.947240,0.972848,0.715149,0.991980,0.992476,0.928810,0.097158,0.026617,0.950756,0.953038
1,46,0.884563,0.476672,0.000969,0.846933,0.742485,0.994332,0.540612,0.412230,0.099950,...,0.515464,0.983204,0.682555,0.993719,0.994706,0.920328,0.979071,0.041499,0.024477,0.863808
2,70,0.950187,0.658466,0.003706,0.696439,0.949114,0.970600,0.607235,0.509221,0.411673,...,0.935586,0.985808,0.730065,0.995348,0.992168,0.945892,0.093498,0.042117,0.930348,0.948664
3,132,0.938729,0.525467,0.002455,0.762707,0.837885,0.890279,0.553451,0.437848,0.049915,...,0.779918,0.947412,0.658828,0.973937,0.982233,0.886768,0.884983,0.145141,0.473806,0.905280
4,200,0.945720,0.569130,0.014231,0.893930,0.744055,0.986870,0.572701,0.508597,0.111417,...,0.502510,0.941061,0.680172,0.984167,0.972880,0.852857,0.290666,0.074808,0.774137,0.900799
